
# Ejercicio agrupaciones y apply

Agrupar los datos por equipo y posición
Calcular la eficiencia de los jugadores ( suma de puntos (PTS), asistencias (AST) y rebotes por juego(TRB) )
Identificar al jugador más eficiente por equipo

Datasets: https://www.kaggle.com/datasets/nathanlauga/nba-games/data

In [5]:
import pandas as pd
import numpy as np
import time

path = '../data/kaggle/nba_data_processed.csv'
df = pd.read_csv(filepath_or_buffer=path, sep= ',', header=0)
df.dropna(how='all',inplace=True)
df.fillna(0,inplace=True)
print(df.shape)
qsna=df.shape[0]-df.isnull().sum(axis=0)
qna=df.isnull().sum(axis=0)
ppna=round(100*(df.isnull().sum(axis=0)/df.shape[0]),2)
aux= {'datos sin NAs en q': qsna, 'Na en q': qna ,'Na en %': ppna}
na=pd.DataFrame(data=aux)
na.sort_values(by='Na en %',ascending=False)

df.head()

(679, 29)


,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Precious Achiuwa,C,23.0,TOR,55.0,12.0,20.7,3.6,7.3,0.485,...,0.702,1.8,4.1,6.0,0.9,0.6,0.5,1.1,1.9,9.2
1,Steven Adams,C,29.0,MEM,42.0,42.0,27.0,3.7,6.3,0.597,...,0.364,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6
2,Bam Adebayo,C,25.0,MIA,75.0,75.0,34.6,8.0,14.9,0.540,...,0.806,2.5,6.7,9.2,3.2,1.2,0.8,2.5,2.8,20.4
3,Ochai Agbaji,SG,22.0,UTA,59.0,22.0,20.5,2.8,6.5,0.427,...,0.812,0.7,1.3,2.1,1.1,0.3,0.3,0.7,1.7,7.9
4,Santi Aldama,PF,22.0,MEM,77.0,20.0,21.8,3.2,6.8,0.470,...,0.750,1.1,3.7,4.8,1.3,0.6,0.6,0.8,1.9,9.0


In [10]:
# Agrupar los datos por equipos
grouped = df.groupby(['Tm', 'Pos'])

